In [ ]:
import pandas as pd

Let's start like before, by reading in our .csv. 
This time, it deals with supemarket sales ans has information on the purchases made, such as the type of the shopper, the gender, the price and quantity of items, the product line, etc. We also get info on the city in which the purchase has been made and the rating (review) of the purchase. Can you think of some research questions that we can pose, knowing we have this information?

In [ ]:
sales = pd.read_csv('data/supermarket_sales.csv')
sales

### Let's explore our dataset a little bit
To get a feel for the dataset, we can take a look at the column names and the first ten entries. 

```head()``` usually shows us the first five rows of the data, but we can also extend that by specifying the number of rows we are interested in as a parameter of the function.

In [ ]:
sales.head(10)

We can also take a random sample from the dataset using the ```sample()``` method. As a parameter, we specify how big of a sample we need.

In [ ]:
sales.sample(5)

We can also check the shape of the DataFrame and the number of unique entries per column. The latter values are considered to be "categorical data", because they sort data in categories or groups. Which columns in our dataset can be considered categorical?

In [ ]:
sales.shape

In [ ]:
sales['Product line'].unique()

We can also check a specific entry (here by using indexing) to see an example entry from our dataset. 

In [ ]:
sales.iloc[200]

In [ ]:
sales.Gender.unique()

We can also check the nr of entries in which the customer was a male:

In [ ]:
sales_male = sales[sales['Gender']=='Male']
sales_male.head(10)

In [ ]:
sales_female = sales[sales['Gender']=='Female']
sales_female.head(10)

In [ ]:
print(f' Purchases made by people identifying as male: {sales_male.shape} \n Purchases made by people identifying as female: {sales_female.shape}')



We can see our dataset is pretty balanced when it comes to the categorical variable of gender.

Or the nr of entries where the total sum of the invoice was less than 100 e.

In [ ]:
sales[sales['Total'] < 100].head(10)


Using shape() we check the number of rows that have the total amount less than a 100 e.

In [ ]:
sales[sales['Total'] < 100].shape

Then, we can do the same for over 100 e.

In [ ]:
sales[sales['Total'] > 100].head(10)

In [ ]:
sales[sales['Total'] > 100].shape

We immidiately can concude by the nr of entries, that most common were the purchases totalling over 100 e (783 vs 205)

In [ ]:
sales['Payment'].unique()

In [ ]:
sales[sales['Payment']=='Cash'].shape


Now, if we wanted to continue playing around, we could do the same for Payment and check the number of entries in each category. We can also try reading that from a plot later on.

### Dataset cleaning

In order to draw any conclusions from our data, our dataset must be clean and balanced.
This can mean different things for different scenarios. At this point, let's just say that we need to figure out how to handle the duplicate entries that might skew our analyses;  what to do with the empty cells and how to change the datatype of the cells.

#### Duplicates

Let's take a look at what the funciton ```duplicated()``` does.

In [ ]:
print(sales.duplicated())

We can also separate the duplicated rows in another dataframe.

In [ ]:
duplicateRows = sales[sales.duplicated()]
duplicateRows

There are multiple ways of dealing with duplicates, e.g.


```
df.drop_duplicates (inplace=True) 

```

which makes sure that we are not creating a new dataframe but that the dupicates are removed from the old one.


We can also use 
```
df = df[df.duplicated(keep='last')]
 
```

which keeps the last occurence of the duplicate

or 

```
df = df[df.duplicated(keep='first')]
```

which keeps the first occurence of the duplicate.

In these cases, we need to store it to a new dataframe variable, otherwise it won't be applied to the dataframe permanently (unlike with the drop_duplicates() function).


In [ ]:
sales[sales.duplicated(keep='last')]

In [ ]:
sales.shape

In [ ]:
sales.drop_duplicates(inplace = True)

In [ ]:
sales.shape

-- We see that the nr of rows diminished.

In [ ]:
sales

We can also uss the parameter ```keep``` with the ```drop_duplicates()``` function (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html).

#### Empty cells

One way to deal with empty cells is to remove rows that contain empty cells. This is usually fine, since data sets can be very big, and removing a few rows will not have a big impact on the result.



In [ ]:
sales[sales.isna().any(axis=1)]


In [ ]:
sales[sales['Branch'].isna()]


We can, for example, fill the empty cell with some other type of value.

In [ ]:
sales["Branch"].fillna('A', inplace = True)

We still have all the other NaN values so let's take care of them. We know we have (at least) one NaN in the Tax 5% column. Let's fill that out by calculating:

In [ ]:
sales_tax = sales.Total * 0.05

In [ ]:
sales["Tax 5%"].fillna(sales_tax, inplace = True)

In [ ]:
sales.iloc[1]

As we see, the 'Tax 5%' column was filled in by the content of the variable ```sales_tax```, i.e. 4.011 (80.22 * 0.05).
We can also use functions like ```mean()```, ```max()```, ```min()``` and  ```mode()``` to fill out the empty values. The first three you are familiar with and ```mode()``` fills out the empty space with the most frequent value in the column:

In [ ]:
mode_payement = sales['Payment'].mode()
mode_payement

In [ ]:
mode_payement = sales['Payment'].mode()[0]
mode_payement

In [ ]:
sales["Payment"].fillna(mode_payement, inplace = True)

Or, we can always just remove the rows with the empty cells, or create a new dataframe that has no empty cells.
We do that through the ```dropna()``` function. If we use it without parameters, it will return a new DataFrame, while if we use the ```inplace = True``` parameter, it will change the original one.

In [ ]:
sales.shape

In [ ]:
sales.dropna(inplace=True)

In [ ]:
sales.shape

In the end, we have lost 7 rows which had NaN values in them, and now our dataframe should be NaN-free:

In [ ]:
sales[sales.isna().any(axis=1)]


#### Wrong type

Cells with data of wrong datatype can make it difficult to analyze data.

To fix it, you have two options: remove the rows, or convert all cells in the columns into the same format.


In [ ]:
sales.dtypes

We have a problem with the quantity. It's a float, but we can never buy two and a half items or smth like that. So we need to cast it to an int. 

In [ ]:
sales.Quantity = sales.Quantity.astype(int)
sales.dtypes

Same with the date and time - they're objects (which stands for a string) and not actually date and time. So let's change that.

In [ ]:
sales.Date = pd.to_datetime(sales.Date)

sales.dtypes

In [ ]:
# Use DataFrame.apply() to convert multiple columns to datetime
sales[['Date','Time']] = sales[['Date','Time']].apply(pd.to_datetime)
sales.dtypes

In [ ]:
sales.head(2)

#### Digression - apply()

apply() is a useful function used to apply (:D) a function along an axis of the DataFrame,  default 0, which is the index (row) axis.
Example:

In [ ]:
def calc_sum(x):
    return x.sum()

data = {
  "x": [50, 40, 30],
  "y": [300, 1112, 42]
}

df = pd.DataFrame(data)

x = df.apply(calc_sum)

print(x)

### Query methods

We already saw that if we find the unique categories in a categorical variable, we can figure out the number of rows depending on a condition (e.g. name of the city). However, sometimes we want to use multiple conditions, and for that we use ```query()```. It automatically returns a new dataframe, but if we want to just update the existing one, we would use ```inplace = True``` as the argument.

In [ ]:
sales['City'].unique()

* purchases made in Yangon

In [ ]:
sales.query('City == "Yangon"')

* purchases made in Yangon in cash

In [ ]:
sales.query('City == "Yangon" & Payment == "Cash"')

* purchases totalling between 1500 and 1000 e

In [ ]:
sales.query('1000 < Total < 1500')

* purchases made in Naypyitaw in cash, that cost less than 50e.

In [ ]:
sales.query('Payment == "Cash" & City =="Naypyitaw" & Total < 50')

### Sum, Max, Min and Average

We can also of course, use functions like ```min()```, ```max()```,```sum()``` and ```mean()``` for basic statistical operations. We can also use ```describe()``` for a glance of the complete statistics of the data in a df or a particular column.

In [ ]:
sales['Total'].sum()

In [ ]:
sales.mean() ['Quantity']

In [ ]:
sales.max()


In [ ]:
sales[sales['Total']== sales.max()['Total']]

In [ ]:
sales[sales['Total']== sales.min()['Total']]

In [ ]:
sales.describe()


### Merging

Let's say we had another .csv that we wanted to add to ours. We would do that in pandas in three ways:
* merge() -  for combining data on common columns or indices
* join() - for combining data on a key column or an index
* concat() - for combining DataFrames across rows or columns


In [ ]:
addition = pd.read_csv('data/addition.csv')
addition

In [ ]:
addition.shape

In [ ]:
dataframes = [sales, addition]
sales = pd.concat(dataframes)
sales.shape

In [ ]:
sales.head(3)

Here the situation is really simple, because we have two dataframes with the same column names. However, the situation can get far more complicated in many cases. I would <b> strongly </b> suggest you read: https://pandas.pydata.org/docs/user_guide/merging.html on all types of merging.

### Groupby

```groupby()``` is used to group the data into categories and apply functions to the categories. It can form groups based on one or more conditions. It can also be used on more than one DataFrames in order to find common categories.

In [ ]:
sales.groupby('City').sum()

In [ ]:
sales.groupby('Customer type').sum()['Total']

In [ ]:
sales.groupby(['Customer type']).agg({'Total': 'sum'})


<b> Digression:</b> 
- ```agg()``` - the method allows you to apply a function or a list of function names to be executed along one of the axis of the DataFrame, default 0, which is the index (row) axis 
     * (https://www.w3schools.com/python/pandas/ref_df_agg.asp)

-  ```count()``` - another useful function - count the number of (not NULL) values in each row                            
      * (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html)
-  ```unstack()``` - used to reshape the given Pandas DataFrame by transposing specified row level to column level 
     *   (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html)


## Let's answer some questions
### Also called "<i>Exploratory data analysis (EDA) </i>"


<div class="alert alert-block alert-info">
<b>Exercises</b>
<p>
    <li>1. Which city has more male shoppers?</li>
    <li>2. Who spends more, men or women (in this particular dataframe)?</li>
    <li>3. Which type of customer spends more, member or non- member?    </li>
    <li>4. Which product line sells the most?</li>
    <li>5. Which product line is popular among men or women?</li>
    <li>6. What day of month makes the highest sales?</li>
    <li>7. What month makes the highest sales?</li>
    <li>8. Find the highest unit price in the product line.</li>
    <li>9. Find the most popular payment method used by customers</li>
    <li>10. Find the payment method that lead to the most amount of money per branch</li>

    
    
    
 
    
    
    
    


</p>
  
</div>



1. Which city has more male shoppers?

In [ ]:
female_shoppers = sales.groupby(['City', 'Gender']).count()['Invoice ID']

In [ ]:
female_shoppers.unstack(level = 0)

If we want to use matplotlib, we only need to add to this line and we get a bar plot:

In [ ]:
female_shoppers.unstack(level = 0).plot(kind='bar')

We can always use our standard libraries for plots, such as matplotlib(https://matplotlib.org/) or seaborn(https://seaborn.pydata.org/index.html). Make sure to investigate them to be able to pick which one you like. I personally, like plotly (https://plotly.com/graphing-libraries/).

To be able to use it, you need to ofc install it. It's available in Anaconda Navigator, but othewise, we can install it in the Anaconda Prompt by typing

```
$ conda install -c plotly plotly=5.14.1
 
```

or in the Terminal 

```
$ pip install plotly==5.14.1


```

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt


In [ ]:
fig = px.bar(sales, x = 'Gender', y = 'Invoice ID',
                     hover_name = 'City',color = 'City')

fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'title': 'city with more male shoppers'})

fig.show()

2. Who spends more, men or women (in this particular dataframe)?


3. Which type of customer spends more, member or non- member?    

4. Which product line sells the most?

5. Which product line is popular among men or women?

6. What day of month makes the highest sales?

In [ ]:
sales['Day'] = sales['Date'].dt.day

In [ ]:
sales.head(3)

In [ ]:
day_sales = sales.groupby('Day').sum()['Total']
day_sales

In [ ]:
day_sales.plot()
plt.show()

In [ ]:
day_sales_df = day_sales.to_frame().reset_index()
day_sales_df.head(2)

In [ ]:
fig = px.line(day_sales_df, x='Day', y="Total")
fig.show()

7. Which month makes the most sales?

8. Find the highest unit price in the product line.


9. Find the most popular payment method used by customers

10. Find the payment method that lead to the most amount of money per branch

### References and literature

- Source of dataset - https://www.kaggle.com/datasets/aungpyaeap/supermarket-sales
- W3Schools Pandas Tutorial - https://www.w3schools.com/python/pandas/default.asp
- Data Science Handbook: https://jakevdp.github.io/PythonDataScienceHandbook/
- more EDA on this dataset - https://www.kaggle.com/code/bharadwajnalla/sales-exploratory-data-analysis
- good tutorial on Meidum - https://medium.com/ds-notes/learning-python-pandas-in-minutes-part-1-basics-f24463da1a18
- good tutorial on GitHub - https://github.com/alod83/data-science